<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/Preparaci%C3%B3nDatos/LimpiezaDataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
#Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Trabajo de Grado/validacionTroncal20230608.csv')

df.columns = ['Station_Access','Day_Group_Type', 'Device', 'Emitter', 'Station','Phase', 'Clearing_Date','Transaction_Date','Peak_Hour','ID_Vehicle','Line','Profile_Name','Card_Number','Operator','Route','Balance_after_transaction','Balance_Before_Transaction','System','Rate_Type','Card_Type','Vehicle_Type','Value','archive']
nombres_columnas = df.dtypes
#Obtener los valores únicos de cada columna
unique_values_day_group_type = df['Day_Group_Type'].unique()
unique_values_fase = df['Phase'].unique()
unique_values_hora_pico = df['Peak_Hour'].unique()
unique_values_sistema = df['System'].unique()
unique_values_tipo_tarjeta = df['Card_Type'].unique()
unique_values_tipo_tarifa =df["Rate_Type"].unique()
unique_values_operador = df ["Operator"].unique()
 # Crear un diccionario con los valores únicos
valores_unicos = {
     'Day_Group_Type': unique_values_day_group_type,
     'Phase': unique_values_fase,
     'Peak_Hour': unique_values_hora_pico,
     'System': unique_values_sistema,
    'Card_Type': unique_values_tipo_tarjeta,
     'Rate_Type': unique_values_tipo_tarifa,
   "Operator": unique_values_operador
 }



Delete6Columns = df.drop(['Day_Group_Type', 'Phase','Peak_Hour','System','Card_Type','Rate_Type','Operator'], axis=1)
Delete6Columns

#Delete6Columns.to_csv('validacionTroncal20230608Col15.csv', index=False)
Delete6Column2= Delete6Columns.drop (['Emitter','Balance_after_transaction','Balance_Before_Transaction','Value','Profile_Name','Card_Number'], axis=1)
Delete6Column2

#Delete6Column2.to_csv('validacionTroncal20230608Col15Col9.csv', index=False)
# Identificando las columnas con valores NaN
nan_columns = Delete6Column2.columns[Delete6Column2.isna().any()].tolist()
nan_columns

Delete3Columns = Delete6Column2.drop (['ID_Vehicle', 'Route', 'Vehicle_Type'], axis=1)
Delete3Columns

#Delete2Columns.to_csv('validacionTroncal20230608Col15Col7.csv', index=False)
Delete2Columns = Delete3Columns.drop (['Line','Clearing_Date','archive'], axis=1)
#Delete2Columns.to_csv('validacionTroncal20230608Col15Col4.csv', index=False)

# Find the 10 most frequent unique values of 'Estacion_Parada'
#top_10_estaciones = Delete2Columns['Station'].value_counts().head(10)

estaciones_seleccionadas = ['(02000) Cabecera Autopista Norte', '(02300) Calle 100','(03000) Portal Suba', '(04000) Cabecera Calle 80','(05000) Portal Américas', '(06000) Portal Eldorado','(07000) Portal Sur','(08000) Portal Tunal','(09000) Cabecera Usme','(10000) Portal 20 de Julio']
#top_10_estaciones = top_estaciones.sort_values(ascending=False).head(10)
#top_10_estaciones

# Filtrar el dataset para incluir solo las 10 estaciones con la mayor cantidad de entradas
filtered_top_10_estaciones_data =Delete2Columns[Delete2Columns['Station'].isin(estaciones_seleccionadas)].copy()
filtered_top_10_estaciones_data


reordered_dataset = filtered_top_10_estaciones_data[['Transaction_Date', 'Station', 'Station_Access', 'Device']].copy()
reordered_dataset


#reordered_dataset.to_csv('validacionTroncal20230608Col15Col410Estaciones.csv', index=False)

reordered_dataset['Transaction_Date'] = pd.to_datetime(reordered_dataset['Transaction_Date'])
reordered_dataset['Day_Week_Number'] = reordered_dataset['Transaction_Date'].dt.dayofweek + 1

co_holidays = holidays.Colombia(years=[2023])
reordered_dataset['Holidays'] = reordered_dataset['Transaction_Date'].apply(lambda x: x in co_holidays)
reordered_dataset['Holidays'] = reordered_dataset['Holidays'].astype(float)

reordered_dataset['Transaction_Date'] = reordered_dataset['Transaction_Date'].astype(str)

# Split the 'Fecha_Transaccion' column into 'Fecha' and 'Hora'
reordered_dataset['Date'] = reordered_dataset['Transaction_Date'].str.split(' ').str[0]
reordered_dataset['Hour'] = reordered_dataset['Transaction_Date'].str.split(' ').str[1]

final_dataset = reordered_dataset[['Date', 'Hour', 'Day_Week_Number', 'Holidays','Station', 'Station_Access', 'Device']]
#final_dataset.to_csv('DatasetConFechaSeparada', index=False)
final_dataset
#final_dataset.to_csv('Dataset2only10top.csv', index=False)
final_dataset = final_dataset.rename(columns={'Hour': 'Time'})

final_dataset['Time'] = pd.to_datetime(final_dataset['Time'], format='%H:%M:%S').dt.time

start_time = pd.to_datetime("03:00:00", format='%H:%M:%S').time()
end_time = pd.to_datetime("23:59:59", format='%H:%M:%S').time()

final_dataset['Range_15_min'] = pd.to_datetime(final_dataset['Date'] + ' ' + final_dataset['Time'].astype(str)).dt.ceil('15T')
grouped_data = final_dataset.groupby(['Range_15_min', 'Day_Week_Number', 'Holidays', 'Station', 'Station_Access', 'Device']).size().reset_index(name='Inputs')

grouped_data
grouped_data['Range_15_min'] = grouped_data['Range_15_min'].astype(str)

# Split the 'Fecha_Transaccion' column into 'Fecha' and 'Hora'
grouped_data['Day'] = grouped_data['Range_15_min'].str.split(' ').str[0]
grouped_data['Hour'] = grouped_data['Range_15_min'].str.split(' ').str[1]

grouped_data15min =  grouped_data[['Day', 'Hour', 'Day_Week_Number', 'Holidays','Station', 'Station_Access', 'Device','Inputs']]
grouped_data15min
#grouped_data15min.to_csv('validacionTroncal20230608Col15Col410EstacionesGroupInputs.csv', index=False)

grouped_data15min['Day'] = pd.to_datetime(grouped_data15min['Day']).copy()
grouped_data15min['Year'] = grouped_data15min['Day'].dt.year
grouped_data15min['Month'] = grouped_data15min['Day'].dt.month
grouped_data15min['Number_Day'] = grouped_data15min['Day'].dt.day

grouped_data15min.drop('Day', axis=1,inplace=True)

grouped_data15min

grouped_data15min['Hour'] = pd.to_datetime(grouped_data15min['Hour'], format='%H:%M:%S').dt.time.copy()
grouped_data15min['Hour_Number'] = grouped_data15min['Hour'].apply(lambda x: x.hour)
grouped_data15min['Minute'] = grouped_data15min['Hour'].apply(lambda x: x.minute)
grouped_data15min['Second'] = grouped_data15min['Hour'].apply(lambda x: x.second)
grouped_data15min.drop(['Hour','Second'], axis=1,inplace=True)
grouped_data15min

ordered_columns = [ 'Year', 'Month', 'Number_Day','Day_Week_Number', 'Holidays', 'Hour_Number', 'Minute', 'Station', 'Station_Access', 'Device','Inputs']
grouped_data15minOrganized = grouped_data15min[ordered_columns].copy()
grouped_data15minOrganized.drop('Year', axis=1,inplace=True)
grouped_data15minOrganized.info()
unique = grouped_data15minOrganized['Station'].unique()
unique
#grouped_data15min.to_csv('validacionTroncal20230608Col15Col410EstacionesGroupInputsDaysHours.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18017 entries, 0 to 18016
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Month            18017 non-null  int64  
 1   Number_Day       18017 non-null  int64  
 2   Day_Week_Number  18017 non-null  int64  
 3   Holidays         18017 non-null  float64
 4   Hour_Number      18017 non-null  int64  
 5   Minute           18017 non-null  int64  
 6   Station          18017 non-null  object 
 7   Station_Access   18017 non-null  object 
 8   Device           18017 non-null  int64  
 9   Inputs           18017 non-null  int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 1.4+ MB


<ipython-input-1-ea1d55f9318a>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_data15min['Day'] = pd.to_datetime(grouped_data15min['Day']).copy()


array(['(05000) Portal Américas', '(07000) Portal Sur',
       '(08000) Portal Tunal', '(09000) Cabecera Usme',
       '(10000) Portal 20 de Julio', '(06000) Portal Eldorado',
       '(02000) Cabecera Autopista Norte', '(02300) Calle 100',
       '(03000) Portal Suba', '(04000) Cabecera Calle 80'], dtype=object)